In [1]:
import random

# Definir el conjunto de funciones
def AND(x, y):
    return x and y

def OR(x, y):
    return x or y

def NOT(x):
    return not x

def NOR(x, y):
    return not (x or y)

# Definir los nodos terminales
nodos_terminales = ['A', 'B', 'C']

# Definir la profundidad máxima de las expresiones
profundidad_maxima = 4
MAX = 4

# Generar un individuo aleatorio
def generar_individuo(profundidad):
    if profundidad == 1:
        return random.choice(nodos_terminales)
    else:
        funcion = random.choice([AND, OR, NOT])
        if funcion == NOT:
            return [funcion.__name__, generar_individuo(profundidad - 1)]
        else:
            return [funcion.__name__, generar_individuo(profundidad - 1), generar_individuo(profundidad - 1)]

# Evaluar un individuo
def evaluar_individuo(individuo, entradas):
    if isinstance(individuo, str):
        return entradas[individuo]
    else:
        nombre_funcion = individuo[0]
        if nombre_funcion == 'AND':
            return AND(evaluar_individuo(individuo[1], entradas), evaluar_individuo(individuo[2], entradas))
        elif nombre_funcion == 'OR':
            return OR(evaluar_individuo(individuo[1], entradas), evaluar_individuo(individuo[2], entradas))
        elif nombre_funcion == 'NOT':
            return NOT(evaluar_individuo(individuo[1], entradas))
        
def mutacion(individuo):
    sub = individuo.copy()
    indice_corte = obtener_indices(sub, MAX)
    max_subarbol = MAX - len(indice_corte)
    subarbol = generar_individuo(profundidad_maxima, max_subarbol)
    nuevo_arbol = insertar_subarbol(individuo, indice_corte, subarbol)
    return nuevo_arbol


def obtener_indices(nested_list, profundidad_maxima, probabilidad_mutacion=0.1):
    '''
    Funcion que regresa los indices del bloque que se modificará en un individuo
    '''
    indices = []
    for i in range(profundidad_maxima):
        if isinstance(nested_list, list):  
            indices.append(random.randint(0, len(nested_list) - 1))
            nested_list = nested_list[indices[i]]
        else:
            break
    return indices[:random.randint(1, len(indices))]

def insertar_subarbol(arbol, indices, subarbol):
    nuevo_arbol = arbol.copy()
    for i in range(len(indices)):
        if indices[i] == 0:
            break

    print("i", i)
    if i == 0:
        nuevo_arbol = subarbol
    elif i == 1:
        nuevo_arbol[indices[0]] = subarbol
    elif i == 2:
        nuevo_arbol[indices[0]][indices[1]] = subarbol
    elif i == 3:       
        nuevo_arbol[indices[0]][indices[1]][indices[2]] = subarbol
    elif i == 4:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]] = subarbol
    elif i == 5:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]] = subarbol
    elif i == 6:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]] = subarbol
    elif i == 7:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]][indices[6]] = subarbol
    elif i == 8:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]][indices[6]][indices[7]] = subarbol
    elif i == 9:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]][indices[6]][indices[7]][indices[8]] = subarbol
    elif i == 10:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]][indices[6]][indices[7]][indices[8]][indices[9]] = subarbol
    return nuevo_arbol

# def mutar_individuo(individuo):
#     nivel_mutacion = random.randint(1, profundidad_maxima - 1)
#     print(f"Nivel de mutación: {nivel_mutacion}")
#     subarbol = generar_individuo(profundidad_maxima-nivel_mutacion)
#     indices = []
#     i = 0
#     while i < nivel_mutacion:
#         rama = random.randint(0, len(individuo_recorrido[i]) - 1)    
#         indices.append(rama)
#         i += 1
    
#     indice_sustitir = index_multi_dimensional_list(individuo, indices)
#     print(f"Indice a sustituir: {indice_sustitir}")
#     return individuo

# def index_multi_dimensional_list(data_list, indices):
#     if not indices:
#         return data_list
#     else:
#         index = indices[0]
#         if index < len(data_list):
#             return index_multi_dimensional_list(data_list[index], indices[1:])
#         else:
#             raise IndexError("Index out of range")


# Probar el algoritmo de programación genética
# individuo = generar_individuo(profundidad_maxima)
# individuo_mutado = mutar_individuo(individuo)
# print(f"Individuo generado: {individuo}")
# print(f"Individuo mutado:   {individuo_mutado}")

# entradas = {'A': True, 'B': False, 'C': True}
# resultado = evaluar_individuo(individuo_mutado, entradas)
# print(f"Resultado evaluado: {resultado}")


In [3]:
def obtener_indices(nested_list, profundidad_maxima, probabilidad_mutacion=0.1):
    '''
    Funcion que regresa los indices del bloque que se modificará en un individuo
    '''
    indices = []
    for i in range(profundidad_maxima):
        if isinstance(nested_list, list):  
            indices.append(random.randint(0, len(nested_list) - 1))
            nested_list = nested_list[indices[i]]
        else:
            break
    return indices[:random.randint(1, len(indices))]

def insertar_subarbol(arbol, indices, subarbol):
    nuevo_arbol = arbol.copy()
    for i in range(len(indices)):
        if indices[i] == 0:
            break

    print("i", i)
    if i == 0:
        nuevo_arbol = subarbol
    elif i == 1:
        nuevo_arbol[indices[0]] = subarbol
    elif i == 2:
        nuevo_arbol[indices[0]][indices[1]] = subarbol
    elif i == 3:       
        nuevo_arbol[indices[0]][indices[1]][indices[2]] = subarbol
    elif i == 4:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]] = subarbol
    elif i == 5:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]] = subarbol
    elif i == 6:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]] = subarbol
    elif i == 7:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]][indices[6]] = subarbol
    elif i == 8:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]][indices[6]][indices[7]] = subarbol
    elif i == 9:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]][indices[6]][indices[7]][indices[8]] = subarbol
    elif i == 10:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]][indices[6]][indices[7]][indices[8]][indices[9]] = subarbol
    return nuevo_arbol

entradas = {'A': True, 'B': False, 'C': True}
nested_list = generar_individuo(3)
indices = obtener_indices(nested_list, 3)
print(indices)
print("arbol original", nested_list)
subarbol = generar_individuo(3)
print("subarbol", subarbol)
arbol_mutado = insertar_subarbol(nested_list, indices, subarbol)
print("arbol mutado", arbol_mutado)

a = evaluar_individuo(arbol_mutado, entradas)
print(a)



[0]
arbol original ['NOT', ['NOT', 'B']]
subarbol ['AND', ['NOT', 'C'], ['NOT', 'B']]
entro
arbol mutado ['AND', ['NOT', 'C'], ['NOT', 'B'], ['NOT', 'B']]
False


In [101]:
a = generar_individuo(4)
indices = obtener_indices(a, 15)
b = generar_individuo(4)


# a =  ['OR', ['NOT', 'B'], ['OR', 'C', 'C']]
# indices = [0, 1]
# b = ['AND', ['NOT', 'C'], ['OR', 'C', 'B']]

indices = [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
a = ['NOT', ['OR', ['AND', 'A', 'A'], ['OR', 'B', 'C']]]
b = ['OR', ['AND', ['AND', 'A', 'B'], ['AND', 'B', 'B']], ['NOT', ['AND', 'C', 'B']]]


print("indices", indices)
print("individuo", a)   
print("subarbol", b)

indices [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
individuo ['NOT', ['OR', ['AND', 'A', 'A'], ['OR', 'B', 'C']]]
subarbol ['OR', ['AND', ['AND', 'A', 'B'], ['AND', 'B', 'B']], ['NOT', ['AND', 'C', 'B']]]


In [102]:
def insertar_subarbol(arbol, indices, subarbol):
    nuevo_arbol = arbol.copy()
    for i in range(len(indices)):
        if indices[i] == 0:
            break

    print("i", i)
    if i == 0:
        nuevo_arbol = subarbol
    elif i == 1:
        nuevo_arbol[indices[0]] = subarbol
    elif i == 2:
        nuevo_arbol[indices[0]][indices[1]] = subarbol
    elif i == 3:       
        nuevo_arbol[indices[0]][indices[1]][indices[2]] = subarbol
    elif i == 4:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]] = subarbol
    elif i == 5:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]] = subarbol
    elif i == 6:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]] = subarbol
    elif i == 7:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]][indices[6]] = subarbol
    elif i == 8:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]][indices[6]][indices[7]] = subarbol
    elif i == 9:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]][indices[6]][indices[7]][indices[8]] = subarbol
    elif i == 10:
        nuevo_arbol[indices[0]][indices[1]][indices[2]][indices[3]][indices[4]][indices[5]][indices[6]][indices[7]][indices[8]][indices[9]] = subarbol
    return nuevo_arbol

print("arbol original", a)
print("subarbol      ", b)
print("indices       ", indices)

nuevo = insertar_subarbol(a, indices, b)
print("nuevo         ", nuevo)




arbol original ['NOT', ['OR', ['AND', 'A', 'A'], ['OR', 'B', 'C']]]
subarbol       ['OR', ['AND', ['AND', 'A', 'B'], ['AND', 'B', 'B']], ['NOT', ['AND', 'C', 'B']]]
indices        [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i 2
nuevo          ['NOT', ['OR', ['AND', 'A', 'A'], ['OR', ['AND', ['AND', 'A', 'B'], ['AND', 'B', 'B']], ['NOT', ['AND', 'C', 'B']]]]]


In [74]:
a[1][1][1] = b

In [103]:
evaluar_individuo(nuevo, entradas)

False

In [96]:
indices = [0,0,0,2,4]
for i in range(len(indices)):
    if indices[i] == 0:
        break
print(i)

0


[1, 2, 3]
